In [1]:
import os, sys
import torch
import numpy as np

from models import build_DABDETR, build_dab_deformable_detr
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops

from PIL import Image
import datasets.transforms as T
import requests, io


vslzr = COCOVisualizer()

def dtd(d,device):
    return {k: v.to(device=device, non_blocking=True) if hasattr(v, 'to') 
               else dtd(v,device) if hasattr(v, 'items') 
               else [dtd(i,device) for i in v] if isinstance(v,list)
               else v for k, v in d.items()}
model_config_path = "model_zoo/DAB_DETR/R50_v2/config.json" # change the path of the model config file
model_checkpoint_path = "model_zoo/DAB_DETR/R50_v2/checkpoint.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

device='cpu'
args = SLConfig.fromfile(model_config_path) 
model, criterion, postprocessors = build_dab_deformable_detr(args)
checkpoint = torch.load(model_checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model'])
model = model.to('cuda')
def dtd(d,device):
    return {k: v.to(device=device, non_blocking=True) if hasattr(v, 'to') 
               else dtd(v,device) if hasattr(v, 'items') 
               else [dtd(i,device) for i in v] if isinstance(v,list)
               else v for k, v in d.items()}

In [2]:
def get_boxes(url):


    response = requests.get(url)
    image_bytes = io.BytesIO(response.content)
    image_init = Image.open(image_bytes).convert("RGB")


    # image
    # transform images
    transform = T.Compose([
        T.RandomResize([800], max_size=1333),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image, _ = transform(image_init, None)
    # predict images
    output = model(image[None].to('cuda'))
    output = dtd(output,'cpu')
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]))[0]
    # visualize outputs
    thershold = 0.35 # set a thershold

    scores = output['scores']
    labels = output['labels']
    boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
    select_mask = scores > thershold

    # box_label = [id2name[int(item)] for item in labels[select_mask]]
    pred_dict = {
        'boxes': boxes[select_mask],
        'size': torch.Tensor([image.shape[1], image.shape[2]]),
        # 'box_label': box_label
    }
    # vslzr.visualize(image, pred_dict, savedir=None)
    W,H = image_init.size

    images = []

    for box in pred_dict['boxes']:
        unnormbbox = box * torch.Tensor([W, H, W, H])
        unnormbbox[:2] -= unnormbbox[2:] / 2
        [bbox_x, bbox_y, bbox_w, bbox_h] = unnormbbox.tolist()
        images.append(image_init.crop((bbox_x,bbox_y,bbox_x+bbox_w,bbox_y+bbox_h)))

    return images

In [3]:
url = 'https://image.shutterstock.com/z/stock-photo-club-sandwich-and-french-fries-on-a-light-wooden-board-next-to-the-potatoes-is-a-cup-of-ketchup-1147730354.jpg'

In [4]:
get_boxes(url)

/home/dima/DAB-DETR/models/dab_deformable_detr/position_encoding.py:53: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
/home/dima/anaconda3/envs/mis/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/dima/DAB-DETR/models/dab_deformable_detr/deformable_transformer.py:489: UserWarning: __floordiv__ is

[<PIL.Image.Image image mode=RGB size=297x315 at 0x7FA5A3FBD5E0>,
 <PIL.Image.Image image mode=RGB size=302x321 at 0x7FA5A3FBDFA0>,
 <PIL.Image.Image image mode=RGB size=307x296 at 0x7FA74C137400>,
 <PIL.Image.Image image mode=RGB size=273x300 at 0x7FA74C137310>,
 <PIL.Image.Image image mode=RGB size=216x216 at 0x7FA74C137A00>]